In [7]:
#!/usr/bin/env python3
"""Show a text-mode spectrogram using live microphone data."""
import argparse
import logging
import numpy as np
import time
import soundfile
import itertools
import IPython.display as ipd
import matplotlib.pyplot as plt
import os
import random
import sounddevice as sd

In [8]:
DURATION=20
gain=10
range=[100,2000]
high=2000
low=100
sequence=5

In [9]:
WORD_TO_TRAIN = ['ba','ta','si','mi','nu','ju']
PATH_TO_TRAIN = os.getcwd()+'/data/train'
PATH_TO_TEST = os.getcwd()+'/data/test'
FILE_PREFIX = "irza"
WORKPATH = PATH_TO_TRAIN

In [54]:
magnitudo=[]
audiodata=[]

def pred(length=1, filename=None, thres=0):
    global cumulated_status, cseq
    try:
        import sounddevice as sd

        #samplerate = sd.query_devices(args.device, 'input')['default_samplerate']
        samplerate = 16000.0

        delta_f = (high - low) / (80 - 1)
        fftsize = np.ceil(samplerate / delta_f).astype(int)
        low_bin = int(np.floor(low / delta_f))

        cumulated_status = sd.CallbackFlags()
        cseq = 0
        def callback(indata, frames, time, status):
            global cumulated_status, cseq, magnitudo
            cumulated_status |= status
            
            cseq += 1
            
            if any(indata):
                magnitude = np.abs(np.fft.rfft(indata[:, 0], n=fftsize))
                magnitude *= gain / fftsize
                audiodata.extend(itertools.chain(indata.tolist()))

                magnitudo.append(magnitude)
                print(type(magnitudo))
                if cseq == 5:
                    cseq = 0
                    #y = model.pred(x=magnitudo)
                    #print(y)
                    print(".")
                    magnitudo=[]
                    
        t0=time.time()
        with sd.InputStream(device=None, channels=1, callback=callback,
                            blocksize=int(samplerate * DURATION / 1000),
                            samplerate=samplerate):
            
            while True:
                t1=time.time()
                if t1-t0 > float(length):
                    if filename!=None: soundfile.write(filename,audiodata,16000)
                    return magnitudo


        if cumulated_status:
            logging.warning(str(cumulated_status))
    except Exception as e:
        print(e)

In [55]:
a = pred(length=1)

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
.
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
.
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
.
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
.
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
.
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
.
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
.
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
.
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
.
<class 'list'>
<class 'list'>
<class 'list'>


(3, 334)

In [43]:
t1 = time.time()
t1-t0

1.8945386409759521